In [44]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn import metrics

data = pd.read_csv('data/googleplaystore.csv')
data = data.dropna()
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 13 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null object
Size              9360 non-null object
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null object
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null object
Android Ver       9360 non-null object
dtypes: float64(1), object(12)
memory usage: 950.8+ KB


In [45]:
def categoryTransform(d):
    CategoryString = d["Category"]
    categoryVal = d["Category"].unique()
    categoryValCount = len(categoryVal)
    category_dict = {}
    for i in range(0, categoryValCount):
        category_dict[categoryVal[i]] = i
    d["Category_i"] = d["Category"].map(category_dict).astype(int)
    
def size_trans(size):
    if 'M' in size:
        x = size[:-1]
        x = float(x)*1000000
        return(x)
    elif 'k' == size[-1:]:
        x = size[:-1]
        x = float(x)*1000
        return(x)
    else:
        return None
    
def sizeTransform(d):
    d["Size"] = d["Size"].map(size_trans)
    data.Size.fillna(method = 'ffill', inplace = True)
    
def installs_trans(inst):
    try:
        x = (inst[:-1].replace(",",""))
        x = int(x)
        #return x
    except ValueError:
        print("Line is corrupt!", x)
        x = 0
    return x
def installsTransform(d):
    d["Installs"] = d["Installs"].map(installs_trans)

def type_trans(types):
    if types == 'Free':
        return 0
    else:
        return 1
def typeTransfor(d):
    d["Type"] = d["Type"].map(type_trans)

def ratingTransform(d):
    RatingL = d["Content Rating"].unique()
    RatingDict = {}
    for i in range(len(RatingL)):
        RatingDict[RatingL[i]] = i
    
    d["Content Rating"] = d["Content Rating"].map(RatingDict).astype(int)

def genreTranform(d):
    GenresL = d.Genres.unique()
    GenresDict = {}
    for i in range(len(GenresL)):
        GenresDict[GenresL[i]] = i
    d["Genres_i"] = d["Genres"].map(GenresDict).astype(int)
    
def price_trans(price):
    if price == '0':
        return 0
    else:
        price = price[1:]
        if price == "veryone":
            price = 0
        price = float(price)
        return price
def priceTransform(d):
    d["Price"] = d["Price"].map(price_trans).astype(float)
    
def review_trans(rev):
    if 'M' in rev:
        x = rev[:-1]
        x = float(x)*1000000
        return(x)
    elif 'k' == rev[-1:]:
        x = rev[:-1]
        x = float(x)*1000
        return(x)
    else:
        return rev
def reviewTrans(d):
    d["Reviews"] = d["Reviews"].map(review_trans).astype(float)

In [46]:
def transformData(d):
    categoryTransform(d)
    sizeTransform(d)
    installsTransform(d)
    typeTransfor(d)
    ratingTransform(d)
    genreTranform(d)
    priceTransform(d)
    reviewTrans(d)
    d.drop(labels = ["Genres","Category","Last Updated","Current Ver", 'Android Ver', 'App'], axis = 1, inplace = True)
    d.info()
    
transformData(data)
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 9 columns):
Rating            9360 non-null float64
Reviews           9360 non-null float64
Size              9360 non-null float64
Installs          9360 non-null int64
Type              9360 non-null int64
Price             9360 non-null float64
Content Rating    9360 non-null int32
Category_i        9360 non-null int32
Genres_i          9360 non-null int32
dtypes: float64(4), int32(3), int64(2)
memory usage: 548.6 KB


,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Category_i,Genres_i
0,4.1,159.0,19000000.0,10000,0,0.0,0,0,0
1,3.9,967.0,14000000.0,500000,0,0.0,0,0,1
2,4.7,87510.0,8700000.0,5000000,0,0.0,0,0,0
3,4.5,215644.0,25000000.0,50000000,0,0.0,1,0,0
4,4.3,967.0,2800000.0,100000,0,0.0,0,0,2


In [64]:
X = [[]]*5
Y = [[]]*5
for field in data.head():
    i = 0
    for value in data.head()[field]:
        if(field == "Rating"):
            Y[i]= value
        else:
            X[i].append(value)
        print(value, i)
        i+=1
        
print(Y)
print(X[0])

4.1 0
3.9 1
4.7 2
4.5 3
4.3 4
159.0 0
967.0 1
87510.0 2
215644.0 3
967.0 4
19000000.0 0
14000000.0 1
8700000.0 2
25000000.0 3
2800000.0 4
10000 0
500000 1
5000000 2
50000000 3
100000 4
0 0
0 1
0 2
0 3
0 4
0.0 0
0.0 1
0.0 2
0.0 3
0.0 4
0 0
0 1
0 2
1 3
0 4
0 0
0 1
0 2
0 3
0 4
0 0
1 1
0 2
0 3
2 4
[4.1, 3.9, 4.7, 4.5, 4.3]
[159.0, 967.0, 87510.0, 215644.0, 967.0, 19000000.0, 14000000.0, 8700000.0, 25000000.0, 2800000.0, 10000, 500000, 5000000, 50000000, 100000, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2]
